In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import spacy
from spacy import displacy

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = (pd.read_csv("../input/stacksample/Questions.csv", nrows=1000000,
                  usecols=['Title', 'Id'],encoding='latin1'))


In [ ]:
titles = iter([_ for _ in df['Title']])

In [ ]:
nlp = spacy.load("en_core_web_sm", disable=["ner"])

In [ ]:
def has_golang(text):
    doc = nlp(text)
    for t in doc:
        if t.lower_ in ["go", "golang"]:
            if t.pos != 'VERB':
                if t.dep_ == "pobj":
                     return True
    
    return False

#g = (doc for doc in titles if has_golang(doc))
#[next(g) for i in range(5)]



In [ ]:
from spacy.matcher import Matcher

In [ ]:
#patterns to be matched
obj_pattern = [{'LOWER': 'objective'},
               {'IS_PUNCT': True, 'OP': '?'},
               {'LOWER': 'c'}]

golang_pattern1 = [{'LOWER' : 'golang'}]
golang_pattern2 = [{'LOWER': 'go',
                   'POS': {'NOT_IN':['VERB']}}]

js_pattern = [{'LOWER': {'IN':['js','javascript']}}]
python_pattern = [{'LOWER': 'python'}]
c_pattern = [{'LOWER' : 'c'}]
net_pattern = [{'LOWER' : 'net'}]
sql_pattern = [{'LOWER' : 'sql'}]
csharp_pattern1 = [{'LOWER':'c'},{'LOWER':'#'}]
csharp_pattern2 = [{'LOWER':'c#'}]
                        
                                  
                                 


In [ ]:
matcher = Matcher(nlp.vocab, validate=True)
matcher.add("obj",None, obj_pattern)
matcher.add("golang",None, golang_pattern1, golang_pattern2)
matcher.add("js",None, js_pattern)
matcher.add("python",None,python_pattern)
matcher.add("c",None,c_pattern)
matcher.add("sql",None,sql_pattern)
matcher.add("csharp",None,csharp_pattern1,csharp_pattern2)




In [ ]:
doc = nlp("Hello I love coding in objective-c and python")

for match_id, start, end in matcher(doc):
    print(doc[start:end])

In [ ]:
titles = iter([x for x in df['Title'] if "python" in x.lower()])

In [ ]:
#do some benchmarking

for i in range(100):
    doc = nlp(next(titles))
    if len(matcher(doc)) == 0:
        print(doc)



In [ ]:
titles = (x for x in df['Title'])
g = (d for d in nlp.pipe(titles) if len(matcher(d)) > 0)

for i in range(20):
    print(next(g))


In [ ]:
df = (pd.read_csv("../input/stacksample/Questions.csv", nrows=100,
                  usecols=['Title', 'Id'],encoding='latin1'))


In [ ]:
df.to_csv('labeled.csv') #i will label the programming languages 

In [ ]:
label_df = (pd.read_csv("../input/labeled1/labeled1.csv")
.assign(Pred=lambda d: [len(matcher(d)) > 0 for d in nlp.pipe(d['Title'])])#see if the condition of matcher is met
.assign(Pred=lambda d: d['Pred'].astype(np.int8))) #convert to 0,1


In [ ]:
label_df[:50]

In [ ]:
mistakes = (label_df
           .loc[lambda d: d['Pred'] == 1]
           .loc[lambda d: d['Label'] == 0]
           )


In [ ]:
mistakes

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
confusion_matrix(label_df['Label'], label_df['Pred']) #there is a few false negatives

In [ ]:
print(classification_report(label_df['Label'], label_df['Pred']))